In [319]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re

In [320]:
df = pd.read_csv("steam_final.csv")
df2 = df.copy()
df.head()

,game_name,price,total_review,positive_percentage,release_year,Indie,Action,Adventure,Casual,Simulation,...,Well-Written,Skiing,Reboot,ATV,BMX,Snowboarding,Action RTS,win,mac,linux
0,Portal 2,999.0,170360,98,2011,0,1,1,0,0,...,0,0,0,0,0,0,0,1,1,1
1,Island Saver,0.0,1213,96,2020,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Stonekeep,999.0,52,100,2017,0,1,1,0,0,...,0,0,0,0,0,0,0,1,1,0
3,Angry Video Game Nerd I & II Deluxe,1499.0,102,98,2020,0,1,1,0,0,...,0,0,0,0,0,0,0,1,1,1
4,The Eden of Grisaia,2999.0,59,98,2017,0,1,1,1,0,...,0,0,0,0,0,0,0,1,0,0


In [321]:
#getting game names and their played hours from steam user profile. 
def get_games(url):
    
    url = "https://steamcommunity.com/id/" + url + "/games/?tab=all"

    response = requests.get(url)

    text = response.text

    text_re = re.findall("(?<=name\":\")(.*?)(?=\")(?<=hours_forever)",text)
    text_t = []
    for k in text_re:
        text_t.append(k.split("\"")[0])
    hours_re = re.findall("(?<=hours_forever\":\")(.*?)(?=\")",text)
    
    hours_re = [float(i) for i in hours_re]

    dataset = pd.DataFrame({'game_name': text_t[:len(hours_re)], 'hours': hours_re})
    dataset2 = dataset.copy()
    average = dataset["hours"].mean()
    
    #dropping games names which are below average 
    for i in range(len(dataset["hours"])):
        if dataset["hours"][i] < average:
            dataset.drop(i, inplace = True)
    
    #returning dropped data and original data
    return dataset["game_name"], dataset2["game_name"]

In [324]:
user_url = input("enter your steam profile name (your account must be public): ")
game_output, game_unfilt = get_games(user_url)
games = game_output.to_numpy()

enter your steam profile name (your account must be public):  Bethlyne


In [326]:
user_index = []
for i in games:
    if i in df2.game_name.values :
        user_index.append(df2.loc[df2['game_name'] == i].index)
        print("{} is found in our database".format(i))
    else:
        print("{} is not found in our database".format(i))

Counter-Strike: Global Offensive is found in our database
Black Desert is not found in our database
Valheim is not found in our database
The Witcher 3: Wild Hunt is not found in our database
Reign Of Kings is found in our database


In [327]:
for i in game_unfiltered:
    df = df[df.game_name != i]

In [328]:
df.drop(columns='game_name', inplace=True)
df.reset_index(drop=True)

,price,total_review,positive_percentage,release_year,Indie,Action,Adventure,Casual,Simulation,Strategy,...,Well-Written,Skiing,Reboot,ATV,BMX,Snowboarding,Action RTS,win,mac,linux
0,0.0,1213,96,2020,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,999.0,52,100,2017,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2,1499.0,102,98,2020,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,1,1
3,2999.0,59,98,2017,0,1,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,999.0,134,96,2015,0,1,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21790,0.0,41,41,2017,1,0,1,0,1,0,...,0,0,0,0,0,0,0,1,1,0
21791,299.0,10,40,2019,1,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
21792,299.0,15,40,2017,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
21793,999.0,120,40,2016,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [329]:
def chosen_platform(x):
    df.drop(df[df[x]==0].index, inplace = True) 

In [330]:
chosen = input("what is your platform")
chosen_platform(chosen)

what is your platform win


In [331]:
df["price"] = df["price"]/100
df_ori = df.copy()
df.drop(columns=["win","mac","linux","VR", "price", "total_review","positive_percentage","release_year"], inplace = True)

In [333]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df = scaler.fit_transform(df)

In [334]:
new_cat_df = []
for i in user_index:
    new_cat_df.append(df[i])   
new_cat_df = [sum(x)/len(new_cat_df )+0.01 for x in zip(*new_cat_df )]
new_cat_df = np.round(new_cat_df)

In [335]:
from sklearn.neighbors import NearestNeighbors

nn_model = NearestNeighbors(n_neighbors=20,algorithm='auto').fit(df)

In [293]:
#we add popularity column that calculated with total_review, release_year, positive_percentage. 
df_ori["popularity"] = (df_ori["total_review"]/(2021-df_ori["release_year"]))*df_ori["positive_percentage"]
columns = ["price","popularity"]

In [295]:
df_ori[columns] = scaler.fit_transform(df_ori[columns])
nn_model2 = NearestNeighbors(n_neighbors=df_ori.shape[0],algorithm='auto').fit(df_ori[columns])

In [297]:
new_df = []
price = []
popularity = []
for i in user_index:
    price.append(df_ori['price'].values[i])
    popularity.append(df_ori["popularity"].values[i])
price = np.mean(price)
popularity = np.mean(popularity)

In [298]:
def get_result():
    output = []
    output.append([price,popularity])
    return output
def get_result_cat():
    output = []
    output.append(new_cat_df)
    return output

In [341]:
distances, indices = nn_model.kneighbors(new_cat_df)
distances2, indices2 = nn_model2.kneighbors(get_result())

In [303]:
def merge_models():
    count = 1
    final = [distances[0][0]]
    for i in indices[0][1:]:
        index = np.where(indices2[0] == i)
        total = float(distances2[0][index] + distances[0][count])
        final.append(total)
        count += 1
    d = {'indices': indices[0], 'distances': final}
    final_df = pd.DataFrame(data = d)
    return final_df

In [304]:
final_df = merge_models()

In [305]:
final_df = final_df.sort_values(by=['distances']).reset_index(drop=True)

In [149]:
count = 1
for i in final_df["indices"]:
    print(str(count) + " " + df2["game_name"][i+1])
    count +=1

1 Survival Zombies The Inverted Evolution
2 Phantom Trigger
3 Caveman Warriors
4 Sirius
5 Sniper Elite
6 Dark Egypt
7 The Underground Man
8 Marlene Betwixt
9 PAYDAY 2
10 INIT.
11 Mindball Play
12 ARM PLANETARY PROSPECTORS EP1 Asteroid Resource Mining
13 Legendary DXP
14 Real Life Battle Royal: It's gonna be an... EPIC game
15 7 Wonders of the Ancient World
16 Double
17 Darwin's bots: Episode 1
18 Outpost Zero
19 The Bionic Commando Pack
20 Imperator: Rome
